In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import os
import wget

/Users/christineliu/.virtualenvs/nvsheng/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
# mv chrome driver from Downloads to Applications 
# mv chrome driver from Downloads to Applications 
chromedriver = "/usr/local/Caskroom/chromedriver/2.41/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver




### keyword 拉取


In [3]:
# 巾帼
website = "http://data.people.com.cn/rmrb/s?qs=%7B%22cds%22%3A%5B%7B%22cdr%22%3A%22AND%22%2C%22cds%22%3A%5B%7B%22fld%22%3A%22title%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22true%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%B7%BE%E5%B8%BC%22%7D%2C%7B%22fld%22%3A%22subTitle%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22false%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%B7%BE%E5%B8%BC%22%7D%2C%7B%22fld%22%3A%22introTitle%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22false%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%B7%BE%E5%B8%BC%22%7D%2C%7B%22fld%22%3A%22contentText%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22true%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%B7%BE%E5%B8%BC%22%7D%5D%7D%5D%7D&tr=A&ss=1&pageNo=1&pageSize=20"
driver = webdriver.Chrome(chromedriver)
driver.get(website)

In [25]:
list_title = []
list_date = []
list_version = []
list_keywords = []

for i in range(1,98):
    titles = driver.find_elements_by_class_name("open_detail_link")
    dates = driver.find_elements_by_class_name("listinfo")
    keywords = driver.find_elements_by_class_name("keywords")
    list_title.extend([title.text for title in titles])
    list_date.extend([date.text for date in dates])
    list_keywords.extend([word.text for word in keywords])
    next_page = driver.find_element_by_xpath('//div/span/a[contains(text(),"下一页")]')
    next_page.click()
    


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/span/a[contains(text(),"下一页")]"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.4 x86_64)


In [26]:
frame = list(zip(list_title,list_date,list_keywords))
frame = pd.DataFrame(frame, columns = ['title','date','keywords'])
frame.tail()

,title,date,keywords
1918,夏衍同志作品中的资产阶级思想,1966年4月1日第5版 【浏览本版】,文章关键词： 人性论 资产阶级民主 思想
1919,全国教书育人楷模候选人事迹简介,2010年8月8日第6版 【浏览本版】,文章关键词： 教师节 学生 教书育人
1920,第四届全国文明城市（区）、文明村镇、文明单位名单,2015年3月1日第7版 【浏览本版】,文章关键词： 北京古船食品有限公司 北京公交客六分公司 国网北京朝阳供电公司
1921,第四届全国文明城市（区）、文明村镇、文明单位名单,2015年3月1日第5版 【浏览本版】,文章关键词： 北京古船食品有限公司 北京公交客六分公司 国网北京朝阳供电公司
1922,第四届全国文明城市（区）、文明村镇、文明单位名单,2015年3月1日第6版 【浏览本版】,文章关键词： 北京古船食品有限公司 北京公交客六分公司 国网北京朝阳供电公司


In [27]:
frame.date = frame.date.apply(lambda x:x.replace("【浏览本版】","").replace("版",""))
frame['section'] = frame.date.apply(lambda x:x.split("第")[1])
frame.head()

,title,date,keywords,section
0,在劳动中书写巾帼华章,2017年5月1日第4,文章关键词： 劳动 文化 中国航天科技集团公司,4
1,“巾帼建功”活动将不断深入,1994年10月7日第3,文章关键词： “巾帼建功” 妇女 就业,3
2,巾帼创业谱新篇（每月聚焦）,1997年12月15日第11,文章关键词： 下岗女工 再就业 创业,11
3,扎实推进“巾帼建功”活动 促进妇女创业就业建功成才,2009年2月19日第3,文章关键词： 妇女 “巾帼建功” 就业,3
4,三总部举行巾帼建功事迹报告会,1991年3月8日第4,文章关键词： 事迹报告会 三总部 解放军总参谋部,4


In [28]:
frame.keywords = frame.keywords.apply(lambda x:x.replace("文章关键词：",""))
frame.keywords = frame.keywords.apply(lambda x: ",".join(x.split()))
frame.head()

,title,date,keywords,section
0,在劳动中书写巾帼华章,2017年5月1日第4,"劳动,文化,中国航天科技集团公司",4
1,“巾帼建功”活动将不断深入,1994年10月7日第3,"“巾帼建功”,妇女,就业",3
2,巾帼创业谱新篇（每月聚焦）,1997年12月15日第11,"下岗女工,再就业,创业",11
3,扎实推进“巾帼建功”活动 促进妇女创业就业建功成才,2009年2月19日第3,"妇女,“巾帼建功”,就业",3
4,三总部举行巾帼建功事迹报告会,1991年3月8日第4,"事迹报告会,三总部,解放军总参谋部",4


In [29]:
frame.date = frame.date.replace("日第.+","",regex=True)
frame.date = frame.date.apply(lambda x:x.replace("年","-").replace("月","-"))
frame.date = pd.to_datetime(frame.date)
frame.head()
frame.to_csv('巾帼人民日报.csv')

In [30]:
# 家庭和谐
website = "http://data.people.com.cn/rmrb/s?qs=%7B%22cds%22%3A%5B%7B%22cdr%22%3A%22AND%22%2C%22cds%22%3A%5B%7B%22fld%22%3A%22title%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22true%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%AE%B6%E5%BA%AD%E5%92%8C%E8%B0%90%22%7D%2C%7B%22fld%22%3A%22subTitle%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22false%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%AE%B6%E5%BA%AD%E5%92%8C%E8%B0%90%22%7D%2C%7B%22fld%22%3A%22introTitle%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22false%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%AE%B6%E5%BA%AD%E5%92%8C%E8%B0%90%22%7D%2C%7B%22fld%22%3A%22contentText%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22true%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E5%AE%B6%E5%BA%AD%E5%92%8C%E8%B0%90%22%7D%5D%7D%5D%7D&tr=A&ss=1&pageNo=1&pageSize=20"
driver = webdriver.Chrome(chromedriver)
driver.get(website)

In [31]:
list_title = []
list_date = []
list_version = []
list_keywords = []

for i in range(1,12):
    titles = driver.find_elements_by_class_name("open_detail_link")
    dates = driver.find_elements_by_class_name("listinfo")
    keywords = driver.find_elements_by_class_name("keywords")
    list_title.extend([title.text for title in titles])
    list_date.extend([date.text for date in dates])
    list_keywords.extend([word.text for word in keywords])
    next_page = driver.find_element_by_xpath('//div/span/a[contains(text(),"下一页")]')
    next_page.click()
    


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/span/a[contains(text(),"下一页")]"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.4 x86_64)


In [32]:
frame = list(zip(list_title,list_date,list_keywords))
frame = pd.DataFrame(frame, columns = ['title','date','keywords'])
frame.tail()

,title,date,keywords
210,最高人民法院工作报告,2018年3月26日第2版 【浏览本版】,文章关键词： 建设 司法 案件
211,最高人民法院工作报告,2017年3月20日第3版 【浏览本版】,文章关键词： 犯罪 司法 案件
212,全国人民代表大会常务委员会工作报告,2016年3月20日第2版 【浏览本版】,文章关键词： 工作报告 宪法 代表
213,全国人民代表大会常务委员会工作报告,2016年3月20日第3版 【浏览本版】,文章关键词： 工作报告 宪法 代表
214,全国人民代表大会常务委员会工作报告,2016年3月20日第1版 【浏览本版】,文章关键词： 工作报告 宪法 代表


In [33]:
frame.date = frame.date.apply(lambda x:x.replace("【浏览本版】","").replace("版",""))
frame['section'] = frame.date.apply(lambda x:x.split("第")[1])
frame.head()

,title,date,keywords,section
0,让孝德在身边扎根（培育和践行社会主义核心价值观）,2014年3月18日第6,文章关键词： 身边扎根 秦皇岛孝文化协会 社会主义,6
1,家庭和谐初探,2008年11月21日第6,文章关键词： 家庭和谐 夫妇 思想,6
2,家庭和谐 是大智慧（全国文明家庭·孝亲敬老）,2017年10月10日第20,文章关键词： 温洪福 家庭和谐 母亲,20
3,发挥妇联作用 促进家庭和谐,2007年1月6日第5,文章关键词： 留守儿童 家庭教育 社区,5
4,推动计划生育基本国策贯彻落实 促进人口长期均衡发展与家庭和谐幸福,2016年5月19日第1,文章关键词： 人口 计划生育 基本国策,1


In [34]:
frame.keywords = frame.keywords.apply(lambda x:x.replace("文章关键词：",""))
frame.keywords = frame.keywords.apply(lambda x: ",".join(x.split()))
frame.head()

,title,date,keywords,section
0,让孝德在身边扎根（培育和践行社会主义核心价值观）,2014年3月18日第6,"身边扎根,秦皇岛孝文化协会,社会主义",6
1,家庭和谐初探,2008年11月21日第6,"家庭和谐,夫妇,思想",6
2,家庭和谐 是大智慧（全国文明家庭·孝亲敬老）,2017年10月10日第20,"温洪福,家庭和谐,母亲",20
3,发挥妇联作用 促进家庭和谐,2007年1月6日第5,"留守儿童,家庭教育,社区",5
4,推动计划生育基本国策贯彻落实 促进人口长期均衡发展与家庭和谐幸福,2016年5月19日第1,"人口,计划生育,基本国策",1


In [35]:
frame.date = frame.date.replace("日第.+","",regex=True)
frame.date = frame.date.apply(lambda x:x.replace("年","-").replace("月","-"))
frame.date = pd.to_datetime(frame.date)
frame.head()
frame.to_csv('家庭和谐人民日报.csv')